In [2]:
# Use Google Colab
use_colab = True
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/685'

    !pip install -r "/content/drive/My Drive/685_Project/requirements.txt"

!pip install sentencepiece

from datasets import load_dataset


import sys
import os
import random
import shutil
import copy
import inspect

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

import numpy as np
import torch
import transformers
import datasets
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tqdm
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/My Drive/685'
/content
     |████████████████████████████████| 298 kB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 34.4 MB/s 
     |████████████████████████████████| 243 kB 43.5 MB/s 
     |████████████████████████████████| 61 kB 475 kB/s 
     |████████████████████████████████| 1.1 MB 35.4 MB/s 
     |████████████████████████████████| 132 kB 45.8 MB/s 
     |████████████████████████████████| 3.3 MB 36.1 MB/s 
     |████████████████████████████████| 596 kB 32.9 MB/s 
     |████████████████████████████████| 895 kB 35.7 MB/s 
     |████████████████████████████████| 160 kB 45.9 MB/s 
     |████████████████████████████████| 192 kB 43.8 MB/s 
     |████████████████████████████████| 271 kB 31.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.

In [3]:
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")


Found device: Tesla K80, n_gpu: 1


In [4]:
# NOT         0 0 0
# OFF UNT     1 1 0
# OFF TIN IND 1 2 1
# OFF TIN GRP 1 2 2
# OFF TIN OTH 1 2 3

In [5]:
import pandas as pd
train = pd.read_csv('/content/drive/My Drive/685_Project/olid_train_v2.csv',sep='\t')
train  = train[train.subtask_a != "NOT"]
#train['subtask_a'] =train['subtask_a'].map({'NOT':0, 'OFF':1, 'UNT':1, 'TIN':2, 'IND':1, 'GRP':2, 'OTH':3, np.nan:0})
train['subtask_b'] =train['subtask_b'].map({'UNT':0, 'TIN':1})
print(len(train["subtask_a"]))
# train["subtask_a"] =  train["subtask_a"].apply(lambda x: list(map(int, x)))


val = train.sample(frac=0.065,replace=False)
final = pd.concat([train, val, val]).drop_duplicates(keep=False)

#sanity checks
assert len(final) == 4400 - len(val)
s1 = pd.merge(val, final, how='inner', on=['id'])
assert len(s1)==0

s1 = pd.merge(final, train, how='inner', on=['id'])
assert len(s1) == len(final)

s1 = pd.merge(val, train, how='inner', on=['id'])
assert len(s1) == len(val)

val.to_csv('olid_val_v2.csv',sep='\t')
final.to_csv('olid_train_v3.csv',sep='\t')

train_dataset = load_dataset('csv', data_files={'train': './olid_train_v3.csv','valid': 'olid_val_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_a_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_a.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_b_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_b.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_c_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_c.csv'}, delimiter="\t", cache_dir='./olid_cache')

4400


Using custom data configuration default-f8e7c00fe483802d


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-f8e7c00fe483802d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


...

Using custom data configuration default-ce0009a1b935ebc5


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-ce0009a1b935ebc5/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(685)

train_dataloader = torch.utils.data.DataLoader(train_dataset['train'], shuffle=True, batch_size=8)
val_dataloader = torch.utils.data.DataLoader(train_dataset['valid'], shuffle=True, batch_size=8)
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], shuffle=True, batch_size=8)


In [7]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', 
    num_labels = 2, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    cache_dir='./roberta_cache'
)
model.to(device) 

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [8]:
batch_size = 99
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
epochs = 5

In [9]:
os.mkdir("trained_model_offenseval_baseline_subtask_b")

In [10]:
import numpy as np
y_pred = []
y_true = []
incorrect = []
# function to get validation accuracy
def get_validation_performance(val_dataloader):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    
    total_correct = 0

    for batch in val_dataloader:
      if len(batch) == 0: continue
      inputs = tokenizer(batch['tweet'],padding='longest',return_tensors="pt").to(device)
      labels = batch['subtask_b'].to(device)
      
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        incorrect.extend([batch["tweet"][index] for index in range(len(batch["tweet"])) if pred_flat[index] != labels_flat[index]])
        y_pred.extend(pred_flat)
        y_true.extend(labels_flat)
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / (len(val_dataloader) * val_dataloader.batch_size)
    return avg_val_accuracy, incorrect, y_pred , y_true

In [11]:
import random

# training loop

# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...

    for batch in train_dataloader:

      if len(batch) == 0: continue
      # print(batch)
      inputs = tokenizer(batch['tweet'],padding='longest',return_tensors="pt").to(device)
      labels = batch['subtask_b'].to(device)
     
      # Clear the previously calculated gradient
      model.zero_grad()        

      # Perform a forward pass (evaluate the model on this training batch).
      outputs = model(**inputs, labels=labels)

      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Update parameters and take a step using the computed gradient.
      optimizer.step()
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc, _, _, _= get_validation_performance(val_dataloader)
    print(f"Validation accuracy: {val_acc}")
    torch.save(model.state_dict(),os.path.join('./trained_model_offenseval_baseline_subtask_b/', 'subtask_b_baseline_{}.pt'.format(epoch_i)))
    
print("")
print("Training complete!")




======== Epoch 1 / 5 ========
Training...
Total loss: 194.99999308213592
Validation accuracy: 0.8819444444444444

======== Epoch 2 / 5 ========
Training...
Total loss: 190.49084348976612
Validation accuracy: 0.8819444444444444

======== Epoch 3 / 5 ========
Training...
Total loss: 191.92467630654573
Validation accuracy: 0.8819444444444444

======== Epoch 4 / 5 ========
Training...
Total loss: 190.15295224636793
Validation accuracy: 0.8819444444444444

======== Epoch 5 / 5 ========
Training...
Total loss: 190.9011639803648
Validation accuracy: 0.8819444444444444

Training complete!


In [12]:
import pandas as pd
test = pd.read_csv('/content/drive/My Drive/685_Project/olid_test_subtask_b.csv',sep='\t')

test['subtask_b'] =test['label'].map({'UNT':0, 'TIN':1})
test.to_csv('olid_test_subtask_b_v2.csv',sep='\t')
test_a_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_b_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], shuffle=True)

Using custom data configuration default-9b566116ccced1df


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-9b566116ccced1df/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
y_pred = []
y_true = []
incorrect = []


In [14]:
acc , incorrect, y_pred , y_true = get_validation_performance(test_dataloader)

In [15]:
from sklearn.metrics import classification_report
report = classification_report(y_true=y_true, y_pred=y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        27
           1       0.89      1.00      0.94       213

    accuracy                           0.89       240
   macro avg       0.44      0.50      0.47       240
weighted avg       0.79      0.89      0.83       240



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
np.savetxt("incorrect_examples_offenseval_subtask_b.csv", 
           incorrect,
           delimiter =", ", 
           fmt ='% s')

In [17]:
from google.colab import files
files.download("incorrect_examples_offenseval_subtask_b.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>